In [1]:
import pandas as pd
import re

# Make functions to extract best model score

Because the model scores have different numbers of columns depending on if it is single label or multi label, format two different types of functions. Test them out on an example to make sure they produce similar results

In [2]:
def get_best_model_single(modelname, file_path_prefix, k_range):
    inner_scores = []
    params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
    
    for k in range(k_range): #Change to 5 if you are using the binary
        inner_df = pd.read_csv(f'{file_path_prefix}{k}.csv')
        inner_df = inner_df.sort_values('F1',ascending=False).reset_index(drop=True)
        inner_scores += inner_df.to_dict('records')
    
    inner_scores = pd.DataFrame.from_dict(inner_scores).fillna(-1)
    best_model = (inner_scores
                  .groupby(params).agg({
                      'F1':'mean',
                      'ROC AUC':'mean',
                      'precision':'mean',
                      'recall':'mean',
                      'accuracy':'mean'
                      }).sort_values('F1',ascending=False).reset_index()).to_dict('records')[0]
    best_model = pd.DataFrame(best_model,index=[modelname])
    
    # re-arrange so hyperparameters are at the end
    best_model = best_model[[c for c in best_model if c not in params] 
        + [c for c in params if c in best_model]]
    
    del inner_scores, inner_df
    return best_model

In [3]:
def get_best_model_multi(modelname, file_path_prefix, k_range): 
    inner_scores = []
    params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
    
    for k in range(k_range): 
        inner_df = pd.read_csv(f'{file_path_prefix}{k}.csv')
        inner_df = inner_df.sort_values('F1 macro', ascending=False).reset_index(drop=True)
        inner_scores += inner_df.to_dict('records')
    
    inner_scores = pd.DataFrame.from_dict(inner_scores).fillna(-1)
    
    #if 'accuracy macro' not in list(inner_scores.columns): # if there is no accuracy macro column set to dummy value
    #    inner_scores['accuracy macro'] = -999 
    best_model = (inner_scores
                .groupby(params).agg({
                    'F1 macro':'mean',
                    'F1 micro':'mean',
                    'F1 weighted':'mean',
                    'ROC AUC macro':'mean',
                    'precision macro':'mean',
                    'recall macro':'mean'
                    }).sort_values('F1 macro',ascending=False).reset_index()).to_dict('records')[0]
    
    best_model = pd.DataFrame(best_model,index=[modelname])
    #best_model.rename(columns={'F1 macro': 'F1', 'ROC AUC macro': 'ROC AUC', 'precision macro':'precision', 'recall macro':'recall', 'accuracy macro':'accuracy'}, inplace=True)
    
    
    ## Get label information
    allLabels = [x for x in inner_scores.columns if 'F1' in x] 
    allLabels.remove('F1 macro')
    allLabels.remove('F1 micro')
    allLabels.remove('F1 weighted')
    allLabels.remove('F1 samples')
    allLabels = [s.replace('F1 - ', '') for s in allLabels]
    
    # Filter to the rows with the best performing configuration
    best_params = best_model[params]
    mask = (inner_scores[list(best_params)] == best_params.iloc[0]).all(axis=1)
    inner_scores_best = inner_scores[mask]
    inner_scores_best
    
    
    ## Start loop to get label scores
    for label in allLabels:
        # get the best model
        best_model_temp = (inner_scores_best
                           .groupby(params).agg({
                               'F1'+' - ' + label:'mean',
                               'ROC AUC'+' - ' + label:'mean',
                               'precision'+' - ' + label:'mean',
                               'recall'+' - ' + label:'mean',
                               'accuracy'+' - ' + label:'mean'
                           }).sort_values('F1'+' - ' + label,ascending=False).reset_index()).to_dict('records')[0]
        best_model_temp = pd.DataFrame(best_model_temp, index=[modelname])

        # take label name out of column names so that everything is general 
        best_model_temp.columns = [s.replace(label, 'label') for s in list(best_model_temp.columns)] 

        # add a column name saying the label
        simpleLabel = label.replace(modelname + '.', '')
        best_model_temp.insert(0, 'label name', simpleLabel) 
        
        # join all together
        best_model = pd.concat([best_model, best_model_temp])
    
    # re-arrange so hyperparameters are at the end
    best_model = best_model[[c for c in best_model if c not in params] 
        + [c for c in params if c in best_model]]
    
    del inner_scores, inner_df, allLabels, best_model_temp
    return best_model

# ORO type

First as a test I ran the model selection proceedure on only 1 fold of the data to estimate the F1 macro of the classifier. It performed poorly, and the individual label performance was heterogeneous:

In [4]:
targetVar = "oro_type" # name of variable
dataFolder = '/homedata/dveytia/Product_1_data'
oro_typev1=get_best_model_multi(f'{targetVar}_v1',f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v1_k', 1)
oro_typev1

,F1 macro,F1 micro,F1 weighted,ROC AUC macro,precision macro,recall macro,label name,F1 - label,ROC AUC - label,precision - label,recall - label,accuracy - label,batch_size,weight_decay,learning_rate,num_epochs,class_weight
oro_type_v1,0.295961,0.705882,0.635972,-1.0,0.384659,0.340196,NaN,NaN,NaN,NaN,NaN,NaN,16,0.0,0.00005,4,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.CDR-BioPump,0.666667,0.977273,1.000000,0.500000,0.990099,16,0.0,0.00005,4,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.MRE-Ocean,0.895105,0.946982,0.941176,0.853333,0.925743,16,0.0,0.00005,3,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.MRE-Bio,0.000000,0.677157,0.000000,0.000000,0.975248,16,0.0,0.00001,2,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.CDR-Cult,0.000000,-1.000000,0.000000,0.000000,1.000000,16,0.0,0.00001,2,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.CDR-OAE,0.222222,0.985075,0.125000,1.000000,0.965347,16,0.0,0.00005,4,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.CDR-BC,0.380952,0.998728,1.000000,0.235294,0.935644,16,0.0,0.00005,4,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.Efficiency,0.000000,0.687032,0.000000,0.000000,0.881188,16,0.0,0.00001,2,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.MRE-Located,0.911565,0.960945,0.930556,0.893333,0.935644,32,0.0,0.00005,4,-1
oro_type_v1,NaN,NaN,NaN,NaN,NaN,NaN,oro_type.CDR-Other,0.000000,-1.000000,0.000000,0.000000,1.000000,16,0.0,0.00001,2,-1


Therefore, I fit the labels sequentially, grouping them by interventions that were conceptually similar, and starting with the best-performing labels. The last resort was to fit a label as a binary classifier when it did not perform well when fit as a multilabel (OAE and biopump)

In [5]:
## This is good -- use this for MRE-Ocean and MRE-Located
targetVar = "oro_type.MRE" # name of variable
v='2_MreLO' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
oro_MRE=get_best_model_multi(f'{targetVar}_v{v}',f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
labelColNames = ['label name',"F1 - label", "ROC AUC - label", "precision - label", "recall - label","accuracy - label", "batch_size","weight_decay","learning_rate","num_epochs","class_weight"]
oro_MRE=oro_MRE.loc[oro_MRE['label name'].isin(['oro_type.MRE-Ocean','oro_type.MRE-Located']),labelColNames]

In [10]:
## This is good -- use predictions for CCS, BC, MRE-Bio and Efficiency labels
targetVar = "oro_type" # name of variable
v='2_notMRE' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
oro_noMRE=get_best_model_multi(f'{targetVar}_v{v}',f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 2)
labelColNames = ['label name',"F1 - label", "ROC AUC - label", "precision - label", "recall - label","accuracy - label", "batch_size","weight_decay","learning_rate","num_epochs","class_weight"]
oro_noMRE=oro_noMRE.loc[oro_noMRE['label name'].isin(['oro_type.CCS','oro_type.CDR-BC','oro_type.MRE-Bio','oro_type.Efficiency']), labelColNames]

Try fitting hard ORO types with binary labels

In [80]:
## For OAE -- good!
targetVar = "oro_type.CDR-OAE" # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
oae_binary=get_best_model_single(f'{targetVar}_v{v}',f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
oae_binary = oae_binary.rename(columns={"F1": "F1 - label", "ROC AUC": "ROC AUC - label", "precision": "precision - label", "recall": "recall - label", "accuracy": "accuracy - label"})
oae_binary.insert(0, "label name", ["oro_type.CDR-OAE"])

In [81]:
## For CDR-BioPump -- good!
targetVar = "oro_type.CDR-BioPump" # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
biopump_binary=get_best_model_single(f'{targetVar}_v{v}',f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
biopump_binary = biopump_binary.rename(columns={"F1": "F1 - label", "ROC AUC": "ROC AUC - label", "precision": "precision - label", "recall": "recall - label", "accuracy": "accuracy - label"})
biopump_binary.insert(0, "label name", ["oro_type.CDR-BioPump"])


In [110]:
# Combine and calculate macro statistics
oro_labels = pd.concat([oro_MRE,oro_noMRE, oae_binary, biopump_binary])
oro_macro = (oro_labels
            .agg({
                "F1 - label": "mean",
                "ROC AUC - label":"mean",
                "precision - label":"mean",
                "recall - label":"mean",
                # "accuracy - label":"mean",
            }
            ))
colNames = [x.replace(" - label"," macro") for x in oro_macro.index]
oro_macro = pd.DataFrame(data = oro_macro).transpose()
oro_macro = oro_macro.set_axis(colNames, axis=1)

oro_type = pd.concat([oro_macro, oro_labels])
oro_type["model_name"] = "oro_type"
oro_type = oro_type.set_index(["model_name"])
oro_type['label name'] = oro_type['label name'].apply(lambda x: re.sub("oro_type.", "", str(x)))
oro_type

,F1 macro,ROC AUC macro,precision macro,recall macro,label name,F1 - label,ROC AUC - label,precision - label,recall - label,accuracy - label,batch_size,weight_decay,learning_rate,num_epochs,class_weight
model_name,,,,,,,,,,,,,,,
oro_type,0.768142,0.945745,0.773661,0.781782,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oro_type,NaN,NaN,NaN,NaN,MRE-Ocean,0.774161,0.931262,0.754704,0.803398,0.897363,16.0,0.0,0.00005,4.0,-1
oro_type,NaN,NaN,NaN,NaN,MRE-Located,0.806463,0.924607,0.741534,0.885640,0.905866,16.0,0.0,0.00001,4.0,"{0: 3.5987654320987654, 1: 3.515151515151515, ..."
oro_type,NaN,NaN,NaN,NaN,CCS,0.718780,0.938736,0.706508,0.734127,0.945437,16.0,0.0,0.00005,4.0,-1.0
oro_type,NaN,NaN,NaN,NaN,CDR-BC,0.841999,0.978452,0.915873,0.785846,0.966095,16.0,0.0,0.00005,3.0,-1.0
oro_type,NaN,NaN,NaN,NaN,MRE-Bio,0.798160,0.975858,0.766046,0.843750,0.979609,16.0,0.0,0.00005,3.0,-1.0
oro_type,NaN,NaN,NaN,NaN,Efficiency,0.797733,0.941134,0.801765,0.803488,0.958256,16.0,0.0,0.00005,4.0,-1.0
oro_type,NaN,NaN,NaN,NaN,CDR-OAE,0.707843,0.948619,0.754630,0.714815,0.974052,16.0,0.0,0.00005,4.0,-1
oro_type,NaN,NaN,NaN,NaN,CDR-BioPump,0.700000,0.927293,0.748232,0.683191,0.973561,32.0,0.0,0.00005,4.0,-1


# Coded variables

Variables coded for articles already relevant for an ORO type (predicted above)

In [88]:
## Primary research 
targetVar = 'primary_research' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
primary_research=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
primary_research

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
primary_research,0.950495,0.887172,0.942464,0.959553,0.910935,16,0.0,0.00001,4,-1


In [89]:
## Implemented -- not sure about the F1 score -- don't do predictions yet
targetVar = 'implemented' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
implemented=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
implemented

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
implemented,0.539742,0.79968,0.621429,0.480795,0.848936,16,0.0,0.00005,2,-1


In [90]:
## Outcome quantitative
targetVar = 'outcome_quantitative' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
outcome_quantitative=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
outcome_quantitative

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
outcome_quantitative,0.886729,0.848679,0.862378,0.913543,0.833936,16,0.0,0.00005,4,"{0: 1, 1: 0.40332640332640335}"


In [91]:
## Outcome effectiveness
targetVar = 'outcome_effectiveness' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
outcome_effectiveness=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
outcome_effectiveness

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
outcome_effectiveness,0.736242,0.759532,0.720541,0.764596,0.711183,32,0.0,0.00005,2,"{0: 1, 1: 0.875}"


In [92]:
## Outcome codis benefit
targetVar = 'outcome_codis_benefit' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
codis_benefit=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
codis_benefit

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
outcome_codis_benefit,0.443967,0.791899,0.401086,0.60681,0.787261,16,0.0,0.00005,3,"{0: 1, 1: 5.958762886597938}"


In [93]:
## Upscaling
targetVar = 'upscaling' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
upscaling=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
upscaling

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
upscaling,0.622749,0.823534,0.609868,0.651725,0.805808,16,0.0,0.00001,4,"{0: 1, 1: 2.9940828402366866}"


In [121]:
## Ecosystem binary
targetVar = 'ecosystem_binary' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
ecosystem_binary=get_best_model_single(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
ecosystem_binary

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
ecosystem_binary,0.878517,0.986774,0.865819,0.894737,0.957079,16,0.0,0.00005,4,-1


In [87]:
## Ecosystem
targetVar = 'ecosystem' # name of variable
v='1' # Version of the script
dataFolder = '/homedata/dveytia/Product_1_data'
ecosystem=get_best_model_multi(f'{targetVar}',
                            f'{dataFolder}/outputs/model_selection/{targetVar}_model_selection_v{v}_k', 3)
ecosystem = ecosystem.drop(columns = ['F1 micro', 'F1 weighted'])
ecosystem

,F1 macro,ROC AUC macro,precision macro,recall macro,label name,F1 - label,ROC AUC - label,precision - label,recall - label,accuracy - label,batch_size,weight_decay,learning_rate,num_epochs,class_weight
ecosystem,0.810341,0.946299,0.845291,0.797888,NaN,NaN,NaN,NaN,NaN,NaN,16,0.0,0.00005,4,-1
ecosystem,NaN,NaN,NaN,NaN,Microalgae,0.826891,0.947246,0.754209,0.925000,0.933855,32,0.0,0.00005,4,"{0: 29.681818181818183, 1: 22.275862068965516,..."
ecosystem,NaN,NaN,NaN,NaN,Macroalgae,0.752381,0.888592,0.969697,0.630952,0.910386,16,0.0,0.00005,4,"{0: 29.681818181818183, 1: 22.275862068965516,..."
ecosystem,NaN,NaN,NaN,NaN,Seagrass,0.952381,0.985119,0.952381,0.952381,0.982906,16,0.0,0.00005,4,-1
ecosystem,NaN,NaN,NaN,NaN,Salt marsh,0.803779,0.932159,0.896296,0.741667,0.934436,16,0.0,0.00005,4,-1
ecosystem,NaN,NaN,NaN,NaN,Mangrove,0.950617,1.000000,1.000000,0.909524,0.966453,16,0.0,0.00005,3,"{0: 29.681818181818183, 1: 22.275862068965516,..."


# Compile all model scores

In [122]:
model_scores_multi = pd.concat([oro_type,ecosystem])
model_scores_multi

,F1 macro,ROC AUC macro,precision macro,recall macro,label name,F1 - label,ROC AUC - label,precision - label,recall - label,accuracy - label,batch_size,weight_decay,learning_rate,num_epochs,class_weight
oro_type,0.768142,0.945745,0.773661,0.781782,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oro_type,NaN,NaN,NaN,NaN,MRE-Ocean,0.774161,0.931262,0.754704,0.803398,0.897363,16.0,0.0,0.00005,4.0,-1
oro_type,NaN,NaN,NaN,NaN,MRE-Located,0.806463,0.924607,0.741534,0.885640,0.905866,16.0,0.0,0.00001,4.0,"{0: 3.5987654320987654, 1: 3.515151515151515, ..."
oro_type,NaN,NaN,NaN,NaN,CCS,0.718780,0.938736,0.706508,0.734127,0.945437,16.0,0.0,0.00005,4.0,-1.0
oro_type,NaN,NaN,NaN,NaN,CDR-BC,0.841999,0.978452,0.915873,0.785846,0.966095,16.0,0.0,0.00005,3.0,-1.0
oro_type,NaN,NaN,NaN,NaN,MRE-Bio,0.798160,0.975858,0.766046,0.843750,0.979609,16.0,0.0,0.00005,3.0,-1.0
oro_type,NaN,NaN,NaN,NaN,Efficiency,0.797733,0.941134,0.801765,0.803488,0.958256,16.0,0.0,0.00005,4.0,-1.0
oro_type,NaN,NaN,NaN,NaN,CDR-OAE,0.707843,0.948619,0.754630,0.714815,0.974052,16.0,0.0,0.00005,4.0,-1
oro_type,NaN,NaN,NaN,NaN,CDR-BioPump,0.700000,0.927293,0.748232,0.683191,0.973561,32.0,0.0,0.00005,4.0,-1
ecosystem,0.810341,0.946299,0.845291,0.797888,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.00005,4.0,-1


In [123]:
model_scores_single = pd.concat([
    primary_research,
    outcome_quantitative,
    outcome_effectiveness,
    upscaling,
    ecosystem_binary
                         ])
model_scores_single

,F1,ROC AUC,precision,recall,accuracy,batch_size,weight_decay,learning_rate,num_epochs,class_weight
primary_research,0.950495,0.887172,0.942464,0.959553,0.910935,16,0.0,0.00001,4,-1
outcome_quantitative,0.886729,0.848679,0.862378,0.913543,0.833936,16,0.0,0.00005,4,"{0: 1, 1: 0.40332640332640335}"
outcome_effectiveness,0.736242,0.759532,0.720541,0.764596,0.711183,32,0.0,0.00005,2,"{0: 1, 1: 0.875}"
upscaling,0.622749,0.823534,0.609868,0.651725,0.805808,16,0.0,0.00001,4,"{0: 1, 1: 2.9940828402366866}"
ecosystem_binary,0.878517,0.986774,0.865819,0.894737,0.957079,16,0.0,0.00005,4,-1


# Join and write all model scores to a csv

In [124]:
best_models_all=pd.concat([model_scores_multi, model_scores_single])

# arrange columns
epsStats = ['label name','F1','ROC AUC','precision','recall','accuracy',
            'F1 macro','ROC AUC macro','precision macro','recall macro',#'accuracy macro',
            'F1 - label','ROC AUC - label','precision - label','recall - label','accuracy - label',
            ]

params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']

best_models_all = best_models_all[epsStats + params] 

# arrange label name at the beginning
# temp_cols=best_models_all.columns.tolist()
# index=best_models_all.columns.get_loc("label name")
# new_cols=temp_cols[index:index+1] + temp_cols[0:index] + temp_cols[index+1:]
# best_models_all=best_models_all[new_cols]


# make the index (model name) its own column and place in position 0
best_models_all['model'] = best_models_all.index
temp_cols=best_models_all.columns.tolist()
index=best_models_all.columns.get_loc("model")
new_cols=temp_cols[index:index+1] + temp_cols[0:index] + temp_cols[index+1:]
best_models_all=best_models_all[new_cols]

# check all the column names
list(best_models_all.columns)
best_models_all.head()

,model,label name,F1,ROC AUC,precision,recall,accuracy,F1 macro,ROC AUC macro,precision macro,...,F1 - label,ROC AUC - label,precision - label,recall - label,accuracy - label,batch_size,weight_decay,learning_rate,num_epochs,class_weight
oro_type,oro_type,nan,NaN,NaN,NaN,NaN,NaN,0.768142,0.945745,0.773661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oro_type,oro_type,MRE-Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.774161,0.931262,0.754704,0.803398,0.897363,16.0,0.0,0.00005,4.0,-1
oro_type,oro_type,MRE-Located,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.806463,0.924607,0.741534,0.885640,0.905866,16.0,0.0,0.00001,4.0,"{0: 3.5987654320987654, 1: 3.515151515151515, ..."
oro_type,oro_type,CCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.718780,0.938736,0.706508,0.734127,0.945437,16.0,0.0,0.00005,4.0,-1.0
oro_type,oro_type,CDR-BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.841999,0.978452,0.915873,0.785846,0.966095,16.0,0.0,0.00005,3.0,-1.0


In [125]:
model_scores_multi.insert(0, 'model', model_scores_multi.index)

In [126]:
model_scores_single.insert(0, 'model', model_scores_single.index)

In [127]:
best_models_all.to_csv(f'/homedata/dveytia/Product_1_data/outputs/model_scores/summary_model_scores_all_2025-07-04.csv', index=False)
model_scores_single.to_csv(f'/homedata/dveytia/Product_1_data/outputs/model_scores/summary_model_scores_single_2025-07-04.csv', index=False)
model_scores_multi.to_csv(f'/homedata/dveytia/Product_1_data/outputs/model_scores/summary_model_scores_multi_2025-07-04.csv', index=False)

In [117]:
oro_typev1.to_csv(f'/homedata/dveytia/Product_1_data/outputs/model_scores/oro_type_v1_model_scores_2025-07-04.csv', index=False)